# TP4 - Machine Learning II 


*  Nombre y Apellido: Lucas Waisten
* Padron: 101908
* Materia: Organizacion de Datos, FIUBA





## Tarea 4: Predicción del score de la review

In [12]:
reviews.sample(5)

,item_id,date_created,comment,rating
94408,1-gram-imported-dutch-mdma-top-quality,2014-05-10,"Extremely disappointed, product substituted w...",0.0
172641,3-5g-8-ball-pseudo-speed-crushed-rock-vac-myle...,2014-07-30,very good : ),5.0
11598,200mg-high-quality-dmt,2014-01-06,fast shpipping!! great communication!!!,5.0
225466,50-1200ug-25i-nbome-blotters,2014-09-04,Great as always. 5/5,5.0
28049,beautiful-creamy-high-grade-pollen-14g-listing...,2014-01-27,im very impresive for the fast delivery to ano...,5.0


¿Qué tan difícil es predecir el score de la review? ¿Cuál es el modelo más sencillo que
puede construirse con resultados aceptables? ¿Cuál es la menor cantidad de datos que
puede usarse para tener datos aceptables?

Modelo utilizados

Baseline: RandomForestClassifier
Red Neuronal utilizando Word Embeddings 

In [3]:
!pip install category_encoders
!pip install wget


/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install gensim

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.1 MB 384 kB/s            
     |████████████████████████████████| 58 kB 347 kB/s            


In [7]:
import pandas as pd
import numpy as np
import category_encoders.utils as util
import gensim
import wget
import gzip
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
drive.mount('/content/drive')
from gensim.models import Word2Vec, KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GRU, Embedding
from keras.models import Model

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
reviews = pd.read_csv('silkroad_reviews.csv')

# Baseline

## Encodiado de los Data Frame

In [14]:
reviews = reviews.loc[:][reviews.rating <=5]

In [31]:
le = {}

Utilizo el OneHotEncoder para definir de forma binaria los features de los rating 

In [15]:
reviews.rating.value_counts()

5.0    297369
0.0     16837
Name: rating, dtype: int64

In [32]:
def encoderOneHot(column,dataFrame,*form):
  if(form[0]=='fit_transform'):
    le[column] = OneHotEncoder(handle_unknown='ignore')
    feature_encoder = le[column].fit_transform(dataFrame[column].astype(str).values.reshape(-1,1))
  else:
    feature_encoder = le[column].transform(dataFrame[column].astype(str).values.reshape(-1,1))
  del(dataFrame[column])
  return dataFrame.join(pd.DataFrame(feature_encoder.todense()).add_prefix(column+'_'))

In [33]:
reviews = encoderOneHot('rating',reviews,'fit_transform')

In [34]:
reviews.rating_0.fillna(0,inplace=True)
reviews.rating_1.fillna(0,inplace=True)

In [35]:
def encoderTfidfVectorizer(column,dataFrame,*form):
  if(form[0]=='fit_transform'):
    le[column] = TfidfVectorizer(max_features=500)
    feature_encoder = le[column].fit_transform(dataFrame[column].astype(str))
  else:
    feature_encoder = le[column].transform(dataFrame[column].astype(str))
  del(dataFrame[column])
  return dataFrame.join(pd.DataFrame(feature_encoder.todense()).add_prefix(column+'_'))

In [36]:
reviews = encoderTfidfVectorizer('comment',reviews,'fit_transform')

In [37]:
limite = reviews.index.size*0.8
top = reviews.index > limite
down = reviews.index <= limite 
validation = reviews[top]
train = reviews[down]

In [39]:
del down
del top

In [40]:
def encoderOrdinal(dataFrame,*form):
  for column in dataFrame.columns:
    if dataFrame[column].dtype==object:
        if(form[0]=='fit_transform'):
          le[column] = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=np.nan)
          dataFrame[column] = le[column].fit_transform(dataFrame[column].astype(str).values.reshape(-1,1))
        else:
          dataFrame[column] = le[column].transform(dataFrame[column].astype(str).values.reshape(-1,1))

In [41]:
encoderOrdinal(train,'fit_transform')
encoderOrdinal(validation,'transform')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

## Imputacion de Nulos

In [42]:
train.fillna(0,inplace=True)
validation.fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


## Entrenamiento del modelo

In [43]:
label_train = train.rating_0
label_validation = validation.rating_0

In [44]:
train

,item_id,date_created,rating_0,rating_1,comment_0,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,comment_8,comment_9,comment_10,comment_11,comment_12,comment_13,comment_14,comment_15,comment_16,comment_17,comment_18,comment_19,comment_20,comment_21,comment_22,comment_23,comment_24,comment_25,comment_26,comment_27,comment_28,comment_29,comment_30,comment_31,comment_32,comment_33,comment_34,comment_35,...,comment_460,comment_461,comment_462,comment_463,comment_464,comment_465,comment_466,comment_467,comment_468,comment_469,comment_470,comment_471,comment_472,comment_473,comment_474,comment_475,comment_476,comment_477,comment_478,comment_479,comment_480,comment_481,comment_482,comment_483,comment_484,comment_485,comment_486,comment_487,comment_488,comment_489,comment_490,comment_491,comment_492,comment_493,comment_494,comment_495,comment_496,comment_497,comment_498,comment_499
0,2875.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1527.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.323517,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.389132,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.304138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9272.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.334046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.601422,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1527.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7671.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461026,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24348,0.0,...,0.0,0.237320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251360,1188.0,291.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251361,1188.0,291.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.312303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251362,1907.0,291.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0

In [45]:
del(train['rating_0'])
del(validation['rating_0'])

In [47]:
search = RandomForestClassifier(random_state=0).fit(train.values,label_train.values)

In [48]:
label_validation

251365    0.0
251366    0.0
251367    0.0
251368    0.0
251369    1.0
         ... 
314204    0.0
314205    0.0
314206    0.0
314207    0.0
314208    0.0
Name: rating_0, Length: 62844, dtype: float64

## Score con las metricas utilizada

In [49]:
roc_auc_score(label_validation,search.predict_proba(validation.values)[:,1])

0.9999518576841614

In [50]:
del le
del train
del validation
del search
del label_train
del label_validation